# import

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [12]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [13]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [15]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [16]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [17]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [18]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [19]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [20]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.4617, train_loss: 2.07716274, train_accuracy: 0.3750, test_accuracy: 0.2777
Epoch: [ 0] [    1/  468] time: 0.5966, train_loss: 1.97578621, train_accuracy: 0.4531, test_accuracy: 0.3684
Epoch: [ 0] [    2/  468] time: 0.7383, train_loss: 1.85593927, train_accuracy: 0.5469, test_accuracy: 0.4706
Epoch: [ 0] [    3/  468] time: 0.8724, train_loss: 1.79366517, train_accuracy: 0.6328, test_accuracy: 0.6040
Epoch: [ 0] [    4/  468] time: 1.0160, train_loss: 1.64878047, train_accuracy: 0.7188, test_accuracy: 0.7039
Epoch: [ 0] [    5/  468] time: 1.1826, train_loss: 1.45448995, train_accuracy: 0.7734, test_accuracy: 0.7402
Epoch: [ 0] [    6/  468] time: 1.3799, train_loss: 1.36005425, train_accuracy: 0.8438, test_accuracy: 0.7650
Epoch: [ 0] [    7/  468] time: 1.5127, train_loss: 1.28934002, train_accuracy: 0.8125, test_accuracy: 0.7841
Epoch: [ 0] [    8/  468] time: 1.6523, 

Epoch: [ 0] [   75/  468] time: 10.8804, train_loss: 0.25651979, train_accuracy: 0.9453, test_accuracy: 0.9191
Epoch: [ 0] [   76/  468] time: 11.0200, train_loss: 0.18567035, train_accuracy: 0.9453, test_accuracy: 0.9174
Epoch: [ 0] [   77/  468] time: 11.1636, train_loss: 0.23076232, train_accuracy: 0.9297, test_accuracy: 0.9155
Epoch: [ 0] [   78/  468] time: 11.3202, train_loss: 0.33231020, train_accuracy: 0.8984, test_accuracy: 0.9174
Epoch: [ 0] [   79/  468] time: 11.4598, train_loss: 0.34940326, train_accuracy: 0.8984, test_accuracy: 0.9196
Epoch: [ 0] [   80/  468] time: 11.6005, train_loss: 0.21057756, train_accuracy: 0.9141, test_accuracy: 0.9219
Epoch: [ 0] [   81/  468] time: 11.7551, train_loss: 0.41451353, train_accuracy: 0.8672, test_accuracy: 0.9228
Epoch: [ 0] [   82/  468] time: 11.8907, train_loss: 0.25112483, train_accuracy: 0.9375, test_accuracy: 0.9236
Epoch: [ 0] [   83/  468] time: 12.0333, train_loss: 0.27991849, train_accuracy: 0.9062, test_accuracy: 0.9236
E

Epoch: [ 0] [  149/  468] time: 21.0377, train_loss: 0.28079146, train_accuracy: 0.9297, test_accuracy: 0.9378
Epoch: [ 0] [  150/  468] time: 21.1594, train_loss: 0.13410288, train_accuracy: 0.9688, test_accuracy: 0.9390
Epoch: [ 0] [  151/  468] time: 21.2891, train_loss: 0.29139900, train_accuracy: 0.9375, test_accuracy: 0.9388
Epoch: [ 0] [  152/  468] time: 21.4165, train_loss: 0.16252694, train_accuracy: 0.9609, test_accuracy: 0.9384
Epoch: [ 0] [  153/  468] time: 21.5448, train_loss: 0.29421860, train_accuracy: 0.9531, test_accuracy: 0.9385
Epoch: [ 0] [  154/  468] time: 21.6715, train_loss: 0.28527406, train_accuracy: 0.9141, test_accuracy: 0.9392
Epoch: [ 0] [  155/  468] time: 21.7950, train_loss: 0.19600084, train_accuracy: 0.9609, test_accuracy: 0.9398
Epoch: [ 0] [  156/  468] time: 21.9111, train_loss: 0.26466805, train_accuracy: 0.9062, test_accuracy: 0.9399
Epoch: [ 0] [  157/  468] time: 22.0427, train_loss: 0.14605899, train_accuracy: 0.9609, test_accuracy: 0.9407
E

Epoch: [ 0] [  223/  468] time: 30.6888, train_loss: 0.22653374, train_accuracy: 0.9375, test_accuracy: 0.9443
Epoch: [ 0] [  224/  468] time: 30.8294, train_loss: 0.15214257, train_accuracy: 0.9453, test_accuracy: 0.9436
Epoch: [ 0] [  225/  468] time: 30.9638, train_loss: 0.17753430, train_accuracy: 0.9375, test_accuracy: 0.9450
Epoch: [ 0] [  226/  468] time: 31.0924, train_loss: 0.13313487, train_accuracy: 0.9609, test_accuracy: 0.9461
Epoch: [ 0] [  227/  468] time: 31.2222, train_loss: 0.18891142, train_accuracy: 0.9375, test_accuracy: 0.9485
Epoch: [ 0] [  228/  468] time: 31.3449, train_loss: 0.26017877, train_accuracy: 0.9297, test_accuracy: 0.9502
Epoch: [ 0] [  229/  468] time: 31.4666, train_loss: 0.19498718, train_accuracy: 0.9375, test_accuracy: 0.9520
Epoch: [ 0] [  230/  468] time: 31.6012, train_loss: 0.17694521, train_accuracy: 0.9297, test_accuracy: 0.9513
Epoch: [ 0] [  231/  468] time: 31.7279, train_loss: 0.07829697, train_accuracy: 0.9844, test_accuracy: 0.9499
E

Epoch: [ 0] [  297/  468] time: 40.2624, train_loss: 0.19489393, train_accuracy: 0.9531, test_accuracy: 0.9554
Epoch: [ 0] [  298/  468] time: 40.3874, train_loss: 0.15559599, train_accuracy: 0.9531, test_accuracy: 0.9559
Epoch: [ 0] [  299/  468] time: 40.5111, train_loss: 0.21585444, train_accuracy: 0.9531, test_accuracy: 0.9562
Epoch: [ 0] [  300/  468] time: 40.6350, train_loss: 0.09263745, train_accuracy: 0.9688, test_accuracy: 0.9566
Epoch: [ 0] [  301/  468] time: 40.7593, train_loss: 0.11700204, train_accuracy: 0.9688, test_accuracy: 0.9549
Epoch: [ 0] [  302/  468] time: 40.8793, train_loss: 0.20811197, train_accuracy: 0.9219, test_accuracy: 0.9540
Epoch: [ 0] [  303/  468] time: 41.0000, train_loss: 0.16597676, train_accuracy: 0.9531, test_accuracy: 0.9538
Epoch: [ 0] [  304/  468] time: 41.1229, train_loss: 0.14576317, train_accuracy: 0.9688, test_accuracy: 0.9540
Epoch: [ 0] [  305/  468] time: 41.2462, train_loss: 0.11701114, train_accuracy: 0.9688, test_accuracy: 0.9553
E

Epoch: [ 0] [  371/  468] time: 49.7611, train_loss: 0.08838002, train_accuracy: 0.9844, test_accuracy: 0.9565
Epoch: [ 0] [  372/  468] time: 49.9037, train_loss: 0.12463295, train_accuracy: 0.9609, test_accuracy: 0.9581
Epoch: [ 0] [  373/  468] time: 50.0513, train_loss: 0.16201736, train_accuracy: 0.9609, test_accuracy: 0.9598
Epoch: [ 0] [  374/  468] time: 50.1929, train_loss: 0.14832740, train_accuracy: 0.9688, test_accuracy: 0.9615
Epoch: [ 0] [  375/  468] time: 50.3405, train_loss: 0.08405106, train_accuracy: 0.9766, test_accuracy: 0.9610
Epoch: [ 0] [  376/  468] time: 50.4702, train_loss: 0.12379012, train_accuracy: 0.9609, test_accuracy: 0.9607
Epoch: [ 0] [  377/  468] time: 50.5992, train_loss: 0.06114796, train_accuracy: 0.9922, test_accuracy: 0.9603
Epoch: [ 0] [  378/  468] time: 50.7233, train_loss: 0.07926900, train_accuracy: 0.9844, test_accuracy: 0.9598
Epoch: [ 0] [  379/  468] time: 50.8470, train_loss: 0.13583398, train_accuracy: 0.9609, test_accuracy: 0.9601
E

Epoch: [ 0] [  445/  468] time: 59.6689, train_loss: 0.13976134, train_accuracy: 0.9453, test_accuracy: 0.9631
Epoch: [ 0] [  446/  468] time: 59.7922, train_loss: 0.09124191, train_accuracy: 0.9609, test_accuracy: 0.9637
Epoch: [ 0] [  447/  468] time: 59.9197, train_loss: 0.11144120, train_accuracy: 0.9609, test_accuracy: 0.9654
Epoch: [ 0] [  448/  468] time: 60.0377, train_loss: 0.13551454, train_accuracy: 0.9453, test_accuracy: 0.9670
Epoch: [ 0] [  449/  468] time: 60.1624, train_loss: 0.08175340, train_accuracy: 0.9844, test_accuracy: 0.9681
Epoch: [ 0] [  450/  468] time: 60.2883, train_loss: 0.06089960, train_accuracy: 0.9844, test_accuracy: 0.9664
Epoch: [ 0] [  451/  468] time: 60.4125, train_loss: 0.10835762, train_accuracy: 0.9766, test_accuracy: 0.9650
Epoch: [ 0] [  452/  468] time: 60.5352, train_loss: 0.22910927, train_accuracy: 0.9219, test_accuracy: 0.9631
Epoch: [ 0] [  453/  468] time: 60.6638, train_loss: 0.07871810, train_accuracy: 0.9688, test_accuracy: 0.9630
E

In [21]:
# test acc = 96.58